In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PytorchAD import AnomalyDetector
from utils import get_feature_map, load_data
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib

# Prepare data

In [ ]:
# Load data
videoinj = load_data("data/videoinj_vector.pkl")
labels = load_data("data/labels.pkl")

# KitNET params
maxAE = 10 #maximum size for any autoencoder in the ensemble layer
FMgrace = 100000 #the number of instances taken to learn the feature mapping (the ensemble's architecture)
ADgrace = 1000000 #the number of instances used to train the anomaly detector (ensemble itself)

v = get_feature_map(videoinj.numpy(), FMgrace, maxAE)

AD = AnomalyDetector(v, device="cpu")
AD = torch.load("model/AD_model.pt")
AD.eval()

# Select threshold

In [ ]:
print("Number of instances: ", len(videoinj))
error_pred = AD(torch.tensor(videoinj[FMgrace+ADgrace+1:])).detach().numpy()
print("Maximum of test error: ", error_pred.max())

#th_list = [0.0, 0.01, 0.03, 0.06,0.064,0.07,0.09,0.1,0.25, 0.3]
th_list = [0.0, 0.01, 0.03,0.04,0.05, 0.06,0.07,0.09,0.1,0.25, 0.3]
rates_of_false_positive = []
rates_of_false_negative = []
accuracy_list = []
benign_samples = np.where(labels[FMgrace+ADgrace+1:] == 0)[0]
mal_samples = np.where(labels[FMgrace+ADgrace+1:] == 1)[0]
for th in th_list:
    pred = (error_pred>th).astype(np.long)
    accuracy_list.append(accuracy_score(pred, labels[FMgrace+ADgrace+1:])*100)
    rates_of_false_positive.append(len(np.where(pred[benign_samples]==1)[0])/len(benign_samples)*100)
    rates_of_false_negative.append(len(np.where(pred[mal_samples]==0)[0])/len(mal_samples)*100)

In [ ]:
# generate axis label
th_str = []
for th in th_list:
    th_str.append(str(th))

font_size = 18
font = {'size':font_size}
matplotlib.rc('font', **font)
fig = plt.figure(figsize = (10, 6))
ax = fig.add_subplot()
ax.plot(range(len(th_list)), rates_of_false_positive, color='r', label="False positive")
ax.plot(range(len(th_list)), rates_of_false_negative, color='b', label="False negative")
ax.plot(range(len(th_list)), accuracy_list, color='g', label="Accuracy")
ax.set_xlabel("Threshold", fontsize=font_size)
ax.set_ylabel("Rate (%)", fontsize=font_size)
ax.set_xticks(range(len(th_list)))
ax.set_xticklabels(labels = th_list, fontsize=font_size)
ax.legend(prop={'size': font_size})
ax.grid(True)
#plt.savefig("result/threshold_select.png", dpi=1000)
plt.savefig("result/videoinj_threshold_select.pdf")

plt.show()


## Plot RMSEs with packet loss

In [ ]:
# plot range

mal_idx = np.where(labels==1)[0]
benign_idx = np.where(labels==0)[0]

start_idx =1200000
end_idx = 2400000
mal_idx = mal_idx[np.where(mal_idx>=start_idx)[0]]
mal_idx = mal_idx[np.where(mal_idx<=end_idx)[0]]
benign_idx = benign_idx[np.where(benign_idx>=start_idx)[0]]
benign_idx = benign_idx[np.where(benign_idx<=end_idx)[0]]

In [ ]:
packet_drop_proba = [0.1, 0.3, 0.5, 0.7]
plt.figure(figsize=(48,36))
font = {'size':60}
matplotlib.rc('font', **font)
for i, p in enumerate(packet_drop_proba):
    
    RMSEs = pickle.load(open("Kitsune/RMSEs_modify_allip_{}.pkl".format(p), "rb"))
    RMSEs = np.array(RMSEs)
    plt.subplot(221+i)
    fig = plt.scatter(benign_idx/1000000,RMSEs[benign_idx],s=18,c='b',label="Benign")
    fig = plt.scatter(mal_idx/1000000,RMSEs[mal_idx],s=18, c='r', label="Malicious")
    plt.yscale("log")
    plt.axhline(y=0.04, linewidth=5)
    plt.axvline(x=1.5, linewidth=5)
    plt.axvline(x=1.8, linewidth=5)
    plt.ylabel("RMSE (log scaled)")
    plt.xlabel(r"Packet number (million)")
    plt.ylim(0,1)
    plt.title("p = {}".format(p))
    
    # jia cu bian kuang
    ax = plt.gca()
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2)
    ax.spines['top'].set_linewidth(2)
    plt.legend()
#plt.savefig("comparison_p.png", dpi=100)
plt.savefig("packet_drop_compare.png", bbox_inche='tight', dpi=100)
